In [3]:
import sqlite3
import pandas as pd

# Conecta ao banco
conn = sqlite3.connect("comissoes.db")
cur = conn.cursor()

In [ ]:
# Cria a tabela
cur.execute("""
CREATE TABLE Vendas (
    Comprador TEXT NOT NULL,
    Vendedor TEXT NOT NULL,
    Data DATE NOT NULL,
    Valor REAL NOT NULL
)
""")

# Insere os dados
dados = [
    ('Leonardo', 'Bruno', '2020-01-01', 200.00),
    ('Leonardo', 'Matheus', '2003-09-27', 1024.00),
    ('Leonardo', 'Lucas', '2006-06-26', 512.00),
    ('Marcos', 'Lucas', '2020-12-17', 100.00),
    ('Marcos', 'Lucas', '2022-03-22', 10.00),
    ('Cinthia', 'Lucas', '2021-03-20', 500.00),
    ('Matheus', 'Bruno', '2007-06-02', 400.00),
    ('Matheus', 'Bruno', '2006-06-26', 400.00),
    ('Matheus', 'Bruno', '2015-06-26', 200.00)
]

cur.executemany("INSERT INTO Vendas VALUES (?, ?, ?, ?)", dados)
conn.commit()

In [10]:
df = pd.read_sql_query("""
SELECT 
    Comprador, 
    Vendedor, 
    Data, 
    Valor
FROM Vendas
""", conn)

print(df)


  Comprador Vendedor        Data   Valor
0  Leonardo    Bruno  2020-01-01   200.0
1  Leonardo  Matheus  2003-09-27  1024.0
2  Leonardo    Lucas  2006-06-26   512.0
3    Marcos    Lucas  2020-12-17   100.0
4    Marcos    Lucas  2022-03-22    10.0
5   Cinthia    Lucas  2021-03-20   500.0
6   Matheus    Bruno  2007-06-02   400.0
7   Matheus    Bruno  2006-06-26   400.0
8   Matheus    Bruno  2015-06-26   200.0


In [46]:
query = """
    SELECT 
        Vendedor, 
        Data, 
        Valor,
        sum(Valor) OVER (PARTITION BY Vendedor ORDER BY Data) AS Total_vendas,
        COUNT(*) OVER (PARTITION BY Vendedor ORDER BY Data) AS Qtde_vendas
    FROM Vendas
    ORDER BY Vendedor, Data
"""

df = pd.read_sql_query(query, conn)
print(df)

  Vendedor        Data   Valor  Total_vendas  Qtde_vendas
0    Bruno  2006-06-26   400.0         400.0            1
1    Bruno  2007-06-02   400.0         800.0            2
2    Bruno  2015-06-26   200.0        1000.0            3
3    Bruno  2020-01-01   200.0        1200.0            4
4    Lucas  2006-06-26   512.0         512.0            1
5    Lucas  2020-12-17   100.0         612.0            2
6    Lucas  2021-03-20   500.0        1112.0            3
7    Lucas  2022-03-22    10.0        1122.0            4
8  Matheus  2003-09-27  1024.0        1024.0            1


In [47]:
query = """
WITH Comissoes AS (
    SELECT 
        Vendedor, 
        Data, 
        Valor,
        sum(Valor) OVER (PARTITION BY Vendedor ORDER BY Data) AS Total_vendas,
        COUNT(*) OVER (PARTITION BY Vendedor ORDER BY Data) AS Qtde_vendas
    FROM Vendas
    ORDER BY Vendedor, Data
    )

    SELECT 
        Vendedor
    FROM Comissoes
    WHERE 
        Total_vendas >= 1024 AND Qtde_vendas <= 3
"""

df = pd.read_sql_query(query, conn)
print(df)

  Vendedor
0    Lucas
1  Matheus
